<a href="https://colab.research.google.com/github/JyotiPrajapati12/LLM_Finetuning_Impression-Generation/blob/main/LLM_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install transformers datasets torch


In [25]:
import pandas as pd

# Load dataset (you'll need to upload your file to Colab)
df = pd.read_csv('/content/impression_300_llm.csv')



In [27]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [28]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permissi

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset


# Load the model and tokenizer after logging in
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")


# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['Report Name'], truncation=True, padding=True)

# Convert your DataFrame to a Hugging Face dataset
train_df = df.iloc[:300]
eval_df = df.iloc[300:]

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Set up Trainer for fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
403 Client Error. (Request ID: Root=1-66fb9467-4bb2dd7b05cea1e40570b656;12f84aca-848e-4e16-8278-3d3c859acdd9)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted and you are not in the authorized list. Visit https://huggingface.co/google/gemma-2b-it to ask for access.

In [ ]:
from transformers import pipeline
from datasets import load_metric

# Load ROUGE metric
rouge = load_metric("rouge")

# Create a pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Evaluate on the evaluation set
def evaluate_model(eval_dataset):
    predictions = []
    references = eval_dataset['Observation']

    for i in range(len(eval_dataset)):
        input_text = eval_dataset['Report Name'][i]
        output = generator(input_text, max_length=100, num_return_sequences=1)
        predictions.append(output[0]['generated_text'])

    # Compute ROUGE
    rouge_result = rouge.compute(predictions=predictions, references=references)
    print("ROUGE Score:", rouge_result)

    return predictions

# Call evaluation
evaluate_model(eval_df)

# Perplexity Calculation
import math
import torch
from torch.nn import CrossEntropyLoss

def compute_perplexity(eval_dataset):
    loss_fct = CrossEntropyLoss()
    total_loss = 0
    total_steps = 0

    for i in range(len(eval_dataset)):
        inputs = tokenizer(eval_dataset['Report Name'][i], return_tensors="pt")
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
        total_loss += loss.item()
        total_steps += 1

    avg_loss = total_loss / total_steps
    perplexity = math.exp(avg_loss)
    print("Perplexity:", perplexity)

compute_perplexity(eval_df)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords and lemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

# Define pre-processing functions
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = text.split()
    # Remove stopwords
    words = [word for word in words if word.lower() not in stop_words]
    # Apply stemming and lemmatization
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    return ' '.join(words)

df['processed_text'] = df['Observation'].apply(preprocess_text)

# Convert text into embeddings
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_text'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(X)

# Extract top 100 word pairs
import numpy as np

def top_similar_word_pairs(matrix, top_n=100):
    pairs = []
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix)):
            pairs.append((i, j, matrix[i, j]))

    # Sort by similarity score
    pairs = sorted(pairs, key=lambda x: x[2], reverse=True)
    return pairs[:top_n]

top_100_pairs = top_similar_word_pairs(similarity_matrix)

# Print the top word pairs
for i, j, sim in top_100_pairs:
    print(f"Pair: {df['Observation'][i]} - {df['Observation'][j]}, Similarity: {sim}")


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Create a graph for visualization
G = nx.Graph()

# Add edges based on top 100 word pairs
for i, j, sim in top_100_pairs:
    G.add_edge(df['Observation'][i], df['Observation'][j], weight=sim)

# Draw the graph
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, k=0.15, iterations=20)
nx.draw(G, pos, with_labels=True, node_size=20, font_size=10)
plt.show()
